# Optical flow Gaussian image denoising

#use_OpenCV = False
use_OpenCV = True
local_debug = True

In [ ]:
import logging
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
#from ipywidgets import *
#import cv2
#import time
#import kernels
#import mrcfile
from collections import namedtuple
from skimage import io as skimage_io

In [ ]:
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)

In [ ]:
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
#from motion_estimation._2D.farneback_OpenCV import Estimator_in_CPU as Estimator
from motion_estimation._2D.farneback_OpenCV import OF_Estimation
#from motion_estimation._2D.project import project
from motion_estimation._2D.project import Projection

In [ ]:
Args = namedtuple("args", "input")
args = Args("barb_0MMPG.png")

In [ ]:
try:
    noisy = skimage_io.imread(args.input)
except FileNotFoundError:
    %run barb_0MMPG.ipynb
    noisy = skimage_io.imread(args.input)

In [ ]:
noisy = skimage_io.imread(args.input)

In [ ]:
#noisy = (255*(noisy - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)

In [ ]:
plt.imshow(noisy, cmap="gray")

In [ ]:
padded_noisy = np.zeros(shape=(noisy.shape[0]+32, noisy.shape[1]+32), dtype=np.uint8)
padded_noisy[16:-16, 16:-16] = noisy
noisy = padded_noisy

In [ ]:
plt.imshow(noisy, cmap="gray")

In [ ]:
def get_gaussian_kernel(sigma=1):
    number_of_coeffs = 3
    number_of_zeros = 0
    while number_of_zeros < 2 :
        delta = np.zeros(number_of_coeffs)
        delta[delta.size//2] = 1
        coeffs = scipy.ndimage.gaussian_filter1d(delta, sigma=sigma)
        number_of_zeros = coeffs.size - np.count_nonzero(coeffs)
        number_of_coeffs += 1
    return coeffs[1:-1]

#sigma = np.array([1.0, 1.0])
sigma = np.array([1.5, 1.5])
#sigma = np.array([2.5, 2.5])
#sigma = np.array([2.0, 2.0])
#sigma = np.array([3.0, 3.0])
kernel = [None]*2
kernel[0] = get_gaussian_kernel(sigma[0])
kernel[1] = get_gaussian_kernel(sigma[1])
print(np.sum(kernel[0]))
plt.plot(kernel[0])
plt.show()

In [ ]:
import denoising.image.OF_gaussian__OpenCV as denoising

In [ ]:
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)
denoiser = denoising.Monochrome_Denoising(logger, N_poly=9, window_side=5)

In [ ]:
denoised = denoiser.filter(noisy, kernel)

In [ ]:
np.min(denoised)

In [ ]:
np.max(denoised)

In [ ]:
denoised = np.clip(denoised, a_min=0, a_max=255)

In [ ]:
denoised = np.round(denoised).astype(np.uint8)

In [ ]:
np.min(denoised)

In [ ]:
np.max(denoised)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy, cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised, cmap="gray")
#axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy, denoised)})")
axs[1].set_title("Denoised")
fig.tight_layout()
plt.show()

In [ ]:
np.mean(noisy)

In [ ]:
np.mean(denoised)

In [ ]:
from skimage.metrics import structural_similarity as ssim
from scipy import stats

In [ ]:
diff = (noisy - denoised).astype(np.uint8)

In [ ]:
plt.imshow(diff, cmap="gray")

In [ ]:
_, N = ssim(noisy, diff, full=True)

In [ ]:
plt.imshow(N, cmap="gray")

In [ ]:
_, P = ssim(noisy, denoised.astype(np.uint8), full=True)

In [ ]:
plt.imshow(P, cmap="gray")

In [ ]:
quality, _ = stats.pearsonr(N.flatten(), P.flatten())

In [ ]:
quality

In [ ]:
plt.imshow(denoised, cmap="gray")
plt.savefig('SPGD_0MMPG_barb.pdf', bbox_inches='tight')